![image info](https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/images/banner_1.png)

# Proyecto 1 - Predicción de precios de vehículos usados

En este proyecto podrán poner en práctica sus conocimientos sobre modelos predictivos basados en árboles y ensambles, y sobre la disponibilización de modelos. Para su desasrrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 1: Predicción de precios de vehículos usados".

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 4. Sin embargo, es importante que avancen en la semana 3 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 4, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/t/b8be43cf89c540bfaf3831f2c8506614).

## Datos para la predicción de precios de vehículos usados

En este proyecto se usará el conjunto de datos de Car Listings de Kaggle, donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como: año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

## Ejemplo predicción conjunto de test para envío a Kaggle

En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

## Rúbrica de Calificación

La calificación estará distribuida de la siguiente manera: 

**Competencia (20 puntos)**

Para cada grupo la calificación será proporcional al lugar que ocupe en el ranking de la competencia. Los grupos que ocupen los primeros 5 puestos obtendrán 20 puntos, a los siguientes 5 grupos se le restarán 2 puntos y así sucesivamente. Es decir: posiciones del 1 al 5 obtendrán 20 puntos, posiciones del 6 al 10 obtendrán 18 puntos, posiciones del 11 a 15 obtendrán 16 puntos y así sucesivamente.

**Informe (80 puntos)**

La calificación del informe se realizará con base en los siguientes criterios (recuerden que deben dar cuenta de cada uno de los aspectos mencionados en los criterios por medio de pantallazos o fracciones de código):

<ins>Preprocesamiento de datos (10 puntos)</ins>

* Los datos de entrenamiento se dividen en datos de entrenamiento y validación. Si decidieron preprocesar los datos (estandarizar, normalizar, imputar valores, etc), estos son correctamente preprocesados al ajustar sobre los datos de entrenamiento (.fit_transform()) y al transformar los datos del set de validación (.transform()). (10 puntos)

<ins>Calibración del modelo (15 puntos)</ins>

* Se calibran los parámetros que se consideren pertinentes del modelo de clasificación seleccionado. (5 puntos)

* Se justifica el método seleccionado de calibración. (5 puntos)

* Se analizan los valores calibrados de cada parámetro y se explica cómo afectan el modelo. (5 puntos)

<ins>Entrenamiento del modelo (15 puntos)</ins>

* Se entrena el modelo de regresión escogido con los datos del set de entrenamiento preprocesados y los parámetros óptimos. (5 puntos)

* Se presenta el desempeño del modelo en los datos de validación con al menos una métrica de desempeño. (5 puntos)

* Se justifica la selección del modelo correctamente. (5 puntos)

<ins>Disponibilización del modelo (30 puntos)</ins>

* Se disponibiliza el modelo en una API alojada en un servicio en la nube. (20 puntos)

* Se hacen las predicciones sobre el valor del automóvil en al menos dos observaciones del set de validación. (10 puntos)

<ins>Conclusiones (10 puntos)</ins>

* Se presentan conclusiones claras y concisas sobre el desarrollo y los resultados del proyecto. (10 puntos)

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

# import all what you need for machine learning
import sklearn
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import RobustScaler
from sklearn import ensemble
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor 
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import BaggingRegressor

from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import StackingRegressor

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFE
import multiprocessing
import math
from statistics import mean
import joblib
import os
os.chdir('..')

# Global settings
n_jobs = -1 # This parameter conrols the parallel processing. -1 means using all processors.
random_state = 42 # This parameter controls the randomness of the data. Using some int value to get same results everytime this code is run.

In [3]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('./Data/dataTrain_carListings.csv')
dataTesting = pd.read_csv('./Data/dataTest_carListings.csv', index_col=0)
data = dataTraining
test = dataTesting

FileNotFoundError: [Errno 2] No such file or directory: './Data/dataTrain_carListings.csv'

## Análisis Exploratorio de Datos

In [ ]:
# Visualización datos de entrenamiento
data.head()

In [ ]:
data.info()

In [ ]:
categorical_cols=data.select_dtypes(include=['object']).columns
categorical_cols

In [ ]:
numerical_cols=data.select_dtypes(exclude=['object']).columns
numerical_cols

In [ ]:
data.describe()

In [ ]:
df=pd.DataFrame(data['Make'].value_counts()).reset_index().rename(columns={'index':'make','Make': 'count'})

In [ ]:
# Cantidad de carros por marca
plt.figure(figsize=(8,8))
plot = sns.barplot(y='make',x='count',data=df)
plot=plt.setp(plot.get_xticklabels(), rotation=45)

In [ ]:
df2=pd.DataFrame(data['State'].value_counts()).reset_index().rename(columns={'index':'state','State': 'count'})

In [ ]:
# Cantidad de carros por state
plt.figure(figsize=(8,8))
plot = sns.barplot(y='state',x='count',data=df2)
plot=plt.setp(plot.get_xticklabels(), rotation=45)

In [ ]:
# Distribución del Precio de los carros
sns.distplot(data['Price'],kde=True)

In [ ]:
# Codificación de las variables categoricas
data['State'] = pd.factorize(data.State)[0]
data['Model'] = pd.factorize(data.Model)[0]
data['Make'] = pd.factorize(data.Make)[0]

test['State'] = pd.factorize(test.State)[0]
test['Model'] = pd.factorize(test.Model)[0]
test['Make'] = pd.factorize(test.Make)[0]

In [ ]:
# df = data[['Price','Mileage','Year']]
df = data
filas = math.ceil(len(df.columns) / 3)
fig, a = plt.subplots(filas,3,figsize=(20, 10))

a = a.ravel()
titles = df.columns

for i, ax in enumerate(a):
    
    if i < len(df.columns):
        ax.hist(df.iloc[:,i])
        ax.set_title(titles[i])
plt.tight_layout()

In [ ]:
# df = data[['Price','Mileage','Year']]
df = data
filas = math.ceil(len(df.columns) / 3)
fig, a = plt.subplots(filas,3,figsize=(20, 10))

a = a.ravel()
titles = df.columns

for i, ax in enumerate(a):
    
    if i < len(df.columns):
        ax.boxplot(df.iloc[:,i])
        ax.set_title(titles[i])
plt.tight_layout()

In [ ]:
print('Cantidad Datos:',len(data))

# Control de Outliers
Q1 = data['Price'].quantile(0.25)
print('Primer Cuartil:', Q1)

Q3 = data['Price'].quantile(0.75)
print('Tercer Cuartil:', Q3)

IQR = Q3 - Q1
print('Rango Intercuartil:', IQR)

Mediana = data['Price'].median()
print('Mediana:', Mediana)

Valor_Minimo = data['Price'].min()
print('Valor Mínimo:', Valor_Minimo)

Valor_Maximo = data['Price'].max()
print('Valor Máximo:', Valor_Maximo)

BI_Calculado = (Q1 - 1.5 * IQR)
print('BI_Calculado: \n', BI_Calculado)

BS_Calculado = (Q3 + 1.5 * IQR)
print('BS_Calculado: \n', BS_Calculado)

In [ ]:
ubicacion_outliers = (data['Price'] < BI_Calculado) | (data['Price'] > BS_Calculado)
outliers = data[ubicacion_outliers]
print('\n Lista de Outliers \n', outliers)

ubicacion_sin_out = (data['Price'] >= BI_Calculado) & (data['Price'] <= BS_Calculado)
data = data[ubicacion_sin_out]
data

In [ ]:
total = len(data)
data['Por_Make'] = 0
for i in data['Make'].unique():
    data.loc[data['Make'] == i,'Por_Make'] = len(data[data['Make'] == i])/total

data['Por_State'] = 0
for i in data['State'].unique():
    data.loc[data['State'] == i,'Por_State'] = len(data[data['State'] == i])/total
    
data.Por_Make.value_counts().hist()

In [ ]:
total = len(test)
test['Por_Make'] = 0
for i in test['Make'].unique():
    test.loc[test['Make'] == i,'Por_Make'] = len(test[test['Make'] == i])/total

test['Por_State'] = 0
for i in test['State'].unique():
    test.loc[test['State'] == i,'Por_State'] = len(test[test['State'] == i])/total
    
test.Por_Make.value_counts().hist()

In [ ]:
sns.heatmap(data.corr())

In [ ]:
# Selección de variables numéricas (X)
# X = data.drop(['Price','State','Model','Make'], axis=1)
# test = test.drop(['State','Model','Make'], axis=1)

X = data.drop(['Price','Make','State','Model'], axis=1)
test = test.drop(['Make','State','Model'], axis=1)

X['YxM'] = X['Year'] * X['Mileage']
test['YxM'] = test['Year'] * test['Mileage']

# Definición variable de interés binaria (y)
y = data['Price']

In [ ]:
# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test usandola función train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=40)

In [ ]:
# estimator = LinearRegression()

# selector = RFE(estimator, n_features_to_select=20, step=5) #Se define step=1, pero lo puede cambiar para que sea más rápido
# selector = selector.fit(X_train, y_train) # esto puede tardar algunos minutos

# selected_vars = X_train.iloc[:,selector.support_].columns
# selected_vars

In [ ]:
# X_train = X_train[selected_vars]
# X_test = X_test[selected_vars]
# test = test[selected_vars]

## Data Standarization

In [ ]:
# define standard scaler
scaler = StandardScaler()
# transform data
X = scaler.fit_transform(X)
test = scaler.fit_transform(test)

In [ ]:
models_scores = [] # To store model scores

def rmse(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    return mean_squared_error(y_test, y_pred, squared= False) # squared= False > returns Root Mean Square Error                  

### Linear Regression

In [ ]:
# linear_regression = make_pipeline(LinearRegression())
# score = rmse(linear_regression)

# models_scores.append(['LinearRegression', score])
# print(f'LinearRegression Score= {score}')

### Lasso

In [ ]:
# lasso = make_pipeline(RobustScaler(), Lasso(alpha=0.0005, random_state= random_state))

# score = rmse(lasso)
# models_scores.append(['Lasso', score])
# print(f'Lasso Score= {score}')

### Elastic Net

In [ ]:
# elastic_net = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio= .9, random_state= random_state))

# score = rmse(elastic_net)
# models_scores.append(['ElasticNet', score])
# print(f'ElasticNet Score= {score}')

## Calibración de Hiperparámetros

In [ ]:
# # Gradient Boosting
# # Validación empleando k-cross-validation y neg_root_mean_squared_error
# # ==============================================================================
# train_scores = []
# cv_scores    = []

# # Valores evaluados
# estimator_range = range(1, 500, 25)

# # Bucle para entrenar un modelo con cada valor de n_estimators y extraer su error
# # de entrenamiento y de k-cross-validation.
# for n_estimators in estimator_range:
    
#     modelo = GradientBoostingRegressor(
#                 n_estimators = n_estimators,
#                 loss         = 'ls',
#                 max_features = 'auto',
#                 random_state = 123
#              )
    
#     # Error de train
#     modelo.fit(X_train, y_train)
#     predicciones = modelo.predict(X = X_train)
#     rmse = mean_squared_error(
#             y_true  = y_train,
#             y_pred  = predicciones,
#             squared = False
#            )
#     train_scores.append(rmse)
    
#     # Error de validación cruzada
#     scores = cross_val_score(
#                 estimator = modelo,
#                 X         = X_train,
#                 y         = y_train,
#                 scoring   = 'neg_root_mean_squared_error',
#                 cv        = 5,
#                 n_jobs    = multiprocessing.cpu_count() - 1,
#              )
#     # Se agregan los scores de cross_val_score() y se pasa a positivo
#     cv_scores.append(-1*scores.mean())
    
# # Gráfico con la evolución de los errores
# fig, ax = plt.subplots(figsize=(6, 3.84))
# ax.plot(estimator_range, train_scores, label="train scores")
# ax.plot(estimator_range, cv_scores, label="cv scores")
# ax.plot(estimator_range[np.argmin(cv_scores)], min(cv_scores),
#         marker='o', color = "red", label="min score")
# ax.set_ylabel("root_mean_squared_error")
# ax.set_xlabel("n_estimators")
# ax.set_title("Evolución del cv-error vs número árboles")
# plt.legend();
# print(f"Valor óptimo de n_estimators: {estimator_range[np.argmin(cv_scores)]}")

In [ ]:
# # Validación empleando k-cross-validation y neg_root_mean_squared_error
# # ==============================================================================
# resultados = {}

# # Valores evaluados
# learning_rates = [0.001, 0.01, 0.1]
# n_estimators   = [10, 20, 100, 200, 300, 400, 500]


# # Bucle para entrenar un modelo con cada combinacion de  learning_rate y n_estimator 
# # y extraer su error de entrenamiento y k-cross-validation.
# for learning_rate in learning_rates:
#     train_scores = []
#     cv_scores    = []
    
#     for n_estimator in n_estimators:
    
#         modelo = GradientBoostingRegressor(
#                     n_estimators  = n_estimator,
#                     learning_rate = learning_rate,
#                     loss          = 'ls',
#                     max_features  = 'auto',
#                     random_state  = 123
#                  )

#         # Error de train
#         modelo.fit(X_train, y_train)
#         predicciones = modelo.predict(X = X_train)
#         rmse = mean_squared_error(
#                 y_true  = y_train,
#                 y_pred  = predicciones,
#                 squared = False
#                )
#         train_scores.append(rmse)

#         # Error de validación cruzada
#         scores = cross_val_score(
#                     estimator = modelo,
#                     X         = X_train,
#                     y         = y_train,
#                     scoring   = 'neg_root_mean_squared_error',
#                     cv        = 3,
#                     n_jobs    = multiprocessing.cpu_count() - 1
#                  )
#         # Se agregan los scores de cross_val_score() y se pasa a positivo
#         cv_scores.append(-1*scores.mean())
        
#     resultados[learning_rate] = {'train_scores': train_scores, 'cv_scores': cv_scores}

# # Gráfico con la evolución de los errores de entrenamiento
# fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 3.84))

# for key, value in resultados.items():
#     axs[0].plot(n_estimators, value['train_scores'], label=f"Learning rate {key}")
#     axs[0].set_ylabel("root_mean_squared_error")
#     axs[0].set_xlabel("n_estimators")
#     axs[0].set_title("Evolución del train error vs learning rate")
    
#     axs[1].plot(n_estimators, value['cv_scores'], label=f"Learning rate {key}")
#     axs[1].set_ylabel("root_mean_squared_error")
#     axs[1].set_xlabel("n_estimators")
#     axs[1].set_title("Evolución del cv-error vs learning rate")
#     plt.legend();

In [ ]:
# # Validación empleando k-cross-validation y neg_root_mean_squared_error
# # ==============================================================================
# train_scores = []
# cv_scores    = []

# # Valores evaluados
# max_depths = [1, 3, 5, 10, 20]

# # Bucle para entrenar un modelo con cada valor de max_depth y extraer su error
# # de entrenamiento y de k-cross-validation.
# for max_depth in max_depths:
    
#     modelo = GradientBoostingRegressor(
#                 n_estimators = 100,
#                 loss         = 'ls',
#                 max_depth    = max_depth,
#                 max_features = 'auto',
#                 random_state = 123
#              )
    
#     # Error de train
#     modelo.fit(X_train, y_train)
#     predicciones = modelo.predict(X = X_train)
#     rmse = mean_squared_error(
#             y_true  = y_train,
#             y_pred  = predicciones,
#             squared = False
#            )
#     train_scores.append(rmse)
    
#     # Error de validación cruzada
#     scores = cross_val_score(
#                 estimator = modelo,
#                 X         = X_train,
#                 y         = y_train,
#                 scoring   = 'neg_root_mean_squared_error',
#                 cv        = 5,
#                 n_jobs    = multiprocessing.cpu_count() - 1
#              )
#     # Se agregan los scores de cross_val_score() y se pasa a positivo
#     cv_scores.append(-1*scores.mean())
    
# # Gráfico con la evolución de los errores
# fig, ax = plt.subplots(figsize=(6, 3.84))
# ax.plot(max_depths, train_scores, label="train scores")
# ax.plot(max_depths, cv_scores, label="cv scores")
# ax.plot(max_depths[np.argmin(cv_scores)], min(cv_scores),
#         marker='o', color = "red", label="min score")
# ax.set_ylabel("root_mean_squared_error")
# ax.set_xlabel("max_depth")
# ax.set_title("Evolución del cv-error vs profundidad árboles")
# plt.legend();
# print(f"Valor óptimo de max_depth: {max_depths[np.argmin(cv_scores)]}")

In [ ]:
# XGB Regresor

In [ ]:
# LGBM Regresor

### Gradient Boosting

In [ ]:
gradient_boosting_regressor= GradientBoostingRegressor(n_estimators=476, learning_rate=0.1,
                                   max_depth=10, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state = random_state)

score = rmse(gradient_boosting_regressor)
models_scores.append(['GradientBoostingRegressor', score])
print(f'GradientBoostingRegressor Score= {score}')

### XGB Regressor

In [ ]:
# xgb_regressor= xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
#                              learning_rate=0.05, max_depth=3, 
#                              min_child_weight=1.7817, n_estimators=2200,
#                              reg_alpha=0.4640, reg_lambda=0.8571,
#                              subsample=0.5213,verbosity=0, nthread = -1, random_state = random_state)
# score = rmse(xgb_regressor)
# models_scores.append(['XGBRegressor', score])
# print(f'XGBRegressor Score= {score}')

### LGBM Regressor

In [ ]:
# lgbm_regressor= lgb.LGBMRegressor(objective='regression',num_leaves=5,
#                               learning_rate=0.05, n_estimators=720,
#                               max_bin = 55, bagging_fraction = 0.8,
#                               bagging_freq = 5, feature_fraction = 0.2319,
#                               feature_fraction_seed=9, bagging_seed=9,
#                               min_data_in_leaf =6, min_sum_hessian_in_leaf = 11,random_state = random_state)

# score = rmse(lgbm_regressor)
# models_scores.append(['LGBMRegressor', score])
# print(f'LGBMRegressor Score= {score}')

In [ ]:
# estimators = [ ('lgbm_regressor', lgbm_regressor) , ('GradientBoostingRegressor', gradient_boosting_regressor),('xgb_regressor', xgb_regressor) ]

# stack = StackingRegressor(estimators=estimators, final_estimator= lasso, cv= 5, n_jobs= n_jobs, passthrough = True)

# stack.fit(X_train, y_train)

# pred = stack.predict(X_test)

# rmse_val = mean_squared_error(y_test, pred, squared= False) # squared= False > returns Root Mean Square Error    
# models_scores.append(['Stacking', rmse_val])
# print(f'rmse= {rmse_val}')

In [ ]:
# Predicción del conjunto de test - acá se genera un número aleatorio como ejemplo
y_pred_test = gradient_boosting_regressor.predict(test)
y_pred = pd.DataFrame(y_pred_test, index=dataTesting.index, columns=['Price'])
y_pred_test

In [ ]:
# Guardar predicciones en formato exigido en la competencia de kaggle
y_pred.to_csv('test_submission.csv', index_label='ID')
y_pred.head()

## Microservicios

In [ ]:
# Exportar modelo a archivo binario .pkl
joblib.dump(gradient_boosting_regressor, 'model_deployment/car_price_reg.pkl', compress=3)

In [ ]:
# Importar modelo y predicción
from model_deployment.m09_model_deployment import predict

# Predicción de probabilidad de que un link sea phishing
predict(0,0,0,0)

# Disponibilizar modelo con Flask

In [ ]:
# Importación librerías
from flask import Flask
from flask_restx import Api, Resource, fields

In [ ]:
# Definición aplicación Flask
app = Flask(__name__)

# Definición API Flask
api = Api(
    app, 
    version='1.0', 
    title='Car Price Prediction API',
    description='Car Price Prediction API')

ns = api.namespace('predict', 
     description='Car Price Regression')

# Definición argumentos o parámetros de la API
parser = api.parser()
# 	Year	Mileage	State	Make
parser.add_argument(
    'YEAR', 
    type=str, 
    required=True, 
    help='Año de Fabricación', 
    location='args')

parser.add_argument(
    'MILEAGE', 
    type=str, 
    required=True, 
    help='Kilometraje', 
    location='args')

parser.add_argument(
    'STATE', 
    type=str, 
    required=True, 
    help='Estado', 
    location='args')

parser.add_argument(
    'MAKE', 
    type=str, 
    required=True, 
    help='Fabricante', 
    location='args')

resource_fields = api.model('Resource', {
    'result': fields.String,
})

In [ ]:
# Definición de la clase para disponibilización
@ns.route('/')
class CarPriceApi(Resource):

    @api.doc(parser=parser)
    @api.marshal_with(resource_fields)
    def get(self):
        args = parser.parse_args()
        
        return {
         "result": predict(args['YEAR','MILEAGE','STATE','MAKE'])
        }, 200

In [ ]:
# Ejecución de la aplicación que disponibiliza el modelo de manera local en el puerto 5000
app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)